<a href="https://colab.research.google.com/github/taruj/LendingClub_CaseStudy/blob/main/LendingClub_CaseStudy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lending Club - Case Study**
### Group Facilitator: Taruj Bhattacharya
### Group Member: Nikhil Shinde

## Exploratory Data Analysis of Lending Club Data to Identify drivers for approving or rejecting loans based on applicant profile.

#### Problem Statement (Business Definition)
When the company receives a loan application, the company has to make a decision for loan approval based on the applicant’s profile. Two types of risks are associated with the bank’s decision:



*   If the applicant **is likely** to repay the loan, then not approving the loan results in a loss (Loss of Business)
*   If the applicant is **not likely** to repay the loan, approving the loan may lead to a financial loss for the company (Default)
